In [14]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

# import _utils # amanda's collected utils

import pandas as pd
import geopandas as gpd
from siuba import *

import shared_utils
import datetime as dt
import numpy as np

import gcsfs

from calitp_data.storage import get_fs
fs = get_fs()

GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/project_prioritization/"

import intake

catalog = intake.open_catalog("./metrics_catalog.yml")

# Safety Metric Demo 
Testing on Watsonville-Santa Cruz multimodal corridor project along SR-1

## Version 1: general project location
This location data comes from the [Rebuilding CA Map](https://rebuildingca.ca.gov/map/)

In [2]:
sample_projects = gpd.read_parquet(f'{GCS_FILE_PATH}nine_sample_projects.parquet')

In [3]:
projloc_sb1 = (sample_projects
               >> filter(_.ct_project_id=="0520000083")
              )

In [4]:
projloc_sb1.explore()

In [5]:
# buffer
projloc_sb1 = projloc_sb1.to_crs(shared_utils.geography_utils.CA_NAD83Albers)
projloc_sb1['b200'] = projloc_sb1.buffer(61)
projloc_sb1 = projloc_sb1.set_geometry('b200')

In [6]:
# load and clip crashes
tims = gpd.read_parquet('gs://calitp-analytics-data/data-analyses/safety_projects/tims_fsi.parquet')

In [7]:
tims_clip = tims.clip(projloc_sb1)
tims_clip = tims_clip.assign(ped_crash = np.where(tims_clip['PEDESTRIAN_ACCIDENT']=="Y",1,0),
                             bike_crash = np.where(tims_clip['BICYCLE_ACCIDENT']=="Y",1,0)
                            )
tims_clip['pedbike_crash'] = tims_clip[["ped_crash", "bike_crash"]].max(axis=1)

In [8]:
tims_clip

,CASE_ID,ACCIDENT_YEAR,COLLISION_DATE,COLLISION_TIME,COLLISION_SEVERITY,PCF_VIOL_CATEGORY,TYPE_OF_COLLISION,MVIW,NUMBER_KILLED,NUMBER_INJURED,PEDESTRIAN_ACCIDENT,BICYCLE_ACCIDENT,LATITUDE,LONGITUDE,POINT_X,POINT_Y,geometry,ped_crash,bike_crash,pedbike_crash
840029,91220793,2020,2020-02-05,25,1,11,G,B,1,0,Y,,36.978001,-121.910202,-121.910202,36.977993,POINT (-169798.400 -113717.958),1,0,1
844341,90505747,2017,2017-07-14,209,2,01,C,E,0,2,,,36.979350,-121.913290,-121.913265,36.979397,POINT (-170067.508 -113556.414),0,0,0
842195,90741755,2018,2018-05-30,1205,2,08,E,I,0,1,,,36.979801,-121.914520,-121.914490,36.979790,POINT (-170175.453 -113510.600),0,0,0
841716,90233219,2016,2016-07-17,334,2,01,E,I,0,1,,,36.981280,-121.916580,-121.916579,36.981283,POINT (-170357.763 -113340.920),0,0,0
840535,8648318,2018,2018-10-06,1946,1,11,G,B,1,0,Y,,NaN,NaN,-121.956947,36.982689,POINT (-173942.247 -113111.362),1,0,1
842080,90625917,2017,2017-12-19,1555,2,08,F,J,0,1,,,36.983800,-121.932370,-121.932367,36.983840,POINT (-171755.140 -113028.200),0,0,0
843108,8387068,2017,2017-06-09,1714,2,03,E,I,0,1,,,NaN,NaN,-121.943946,36.983688,POINT (-172784.556 -113024.060),0,0,0
844472,90578468,2017,2017-10-15,1630,2,01,E,I,0,1,,,36.983730,-121.932800,-121.932794,36.983871,POINT (-171793.055 -113024.043),0,0,0
843883,90249267,2016,2016-02-20,2300,1,18,G,B,1,0,Y,,36.983630,-121.952230,-121.952230,36.983635,POINT (-173520.883 -113014.873),1,0,1
842313,90874386,2018,2018-11-13,1820,2,03,C,C,0,1,,,36.983799,-121.936401,-121.936401,36.983910,POINT (-172113.514 -113013.202),0,0,0


### Demo Crash Reduction Factors:
* Transit signal priority: 14% [cmf clearinghouse reference](https://www.cmfclearinghouse.org/detail.php?facid=11233)
* Mode separation
* crosswalks/flashing beacons: 15% [ref](https://www.cmfclearinghouse.org/detail.php?facid=2917)
* Aux lane: 23% [ref](https://www.cmfclearinghouse.org/detail.php?facid=3899)

In [9]:
# combine crash reduction factors - toy example
# reference: https://www.cmfclearinghouse.org/collateral/Combining_Multiple_CMFs_Final.pdf 
# CCRFi = 1 – [(1-CRF1i)*(1-CRF2i)*(1-CRF3i)] 

CRF = 1-((1-0.14)*(1-0.15)*(1-0.23))

In [10]:
CRF

0.43713

In [11]:
CRF*len(tims_clip)

6.99408

## Version 2: mode separated location
This location data is hand-drawn

In [15]:
# read in shapefiles
projloc_bikeped = catalog.shp_demoproj_bikeped.read()

In [18]:
projloc_bikeped = projloc_bikeped.to_crs(shared_utils.geography_utils.CA_NAD83Albers)

In [19]:
projloc_bikeped.explore()